# Hedging risks

The point of this assignment is to assess the risks of using futures to hedge exposure to jet fuel prices
for an airline and, more generally, to see how futures may not align with asset risk.

One of the largest expenses for an airline is jet fuel, and futures are one possible
way to manage the risk that rising prices adversely affect the balance sheet. Suppose the airline
has already done demand modeling and determined that it will need about 100 million gallons of
jet fuel for this month (November 2025). Suppose it is January 2024, and they must decide
how to hedge the price risk on 100 million gallons of jet fuel.

There are so many pieces to this that a complete analysis could be much longer than
one homework assignment. The instructions leave room for your interpretation and preference of
what a good approach to each part is, but you should not feel the need to provide quantitative
justification for each decision that must be made. Rather I hope that you just take the opportunity
to work through the process to see the components of risk, their relative scale, and the quantitative
and modeling decisions needed.

NOTE/WARNING: The splicing methods in `finm37000` do not handle splicing when roll dates do
not have data for both contracts. Databento `ohlcv-1d` data rolls on UTC dates and may contain
weekend data for one contract and not the other. If you would like to use that data and these
splicing methods, you should probably explicitly force the use of weekday data only.
Other data sources and splicing methods are acceptable too and may not face this issue.


In [2]:
import os
import sys

# From root/notebooks -> go one level up to root
ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

ROOT

'/Users/adithsrinivasan/Documents/GitHub/finm37000-2025'

In [ ]:
import datetime
import databento as db
import pathlib
import pandas as pd
import numpy as np
import statsmodels.api as sm

from src.finm37000 import get_databento_api_key, temp_env
from src.finm37000.futures import filter_legs
from src.finm37000.constant_maturity import get_roll_spec, constant_maturity_splice


In [25]:
Q_J = 100_000_000  # gallons
Q_HO = 42_000  # gallons per contract
CUTOFF_DATE = "2024-01-02"
CME_DATASET = "GLBX.MDP3"

In [32]:
with temp_env(DATABENTO_API_KEY=get_databento_api_key(f"{ROOT}/.databento_api_key")):
    client = db.Historical()

1. A future on jet fuel would provide the least product basis risk, but there is no jet fuel future on NYMEX. If you search among refined
products though, you may find the jet fuel/heating oil spread future: https://www.cmegroup.com/markets/energy/refined-products/gulf-coast-jet-fuel-vs-nymex-no-2-heating-oil-platts-spread-swap.contractSpecs.html

This suggests one popular choice: using heating oil as a proxy for jet fuel, then potentially using the spread future to manage
the difference between them.

`How many heating oil futures should the airline use to hedge their jet fuel risk? Your answer should account for the contract size,
information that would have been available in January 2024, and how well heating oil has tracked jet fuel prices.

Note that petroleum product spot prices can be found from the US Energy Information Administration https://www.eia.gov/dnav/pet/pet_pri_spt_s1_d.htm

In [16]:
jet_path = pathlib.Path("../data/EER_EPJK_PF4_RGC_DPGd.xls")

print(jet_path)

jet_raw = pd.read_excel(
    jet_path,
    sheet_name="Data 1"
)

print(jet_raw.columns)

jet = (
    jet_raw
    .rename(columns={
        "U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB (Dollars per Gallon)": "jet_price"
    })
    .dropna()
)

jet["Date"] = pd.to_datetime(jet["Date"])
jet = jet.set_index("Date").sort_index()

../data/EER_EPJK_PF4_RGC_DPGd.xls
Index(['Date', 'U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB (Dollars per Gallon)'], dtype='object')


In [41]:
long_history_start = datetime.date(2018, 1, 1)

# All HO futures definitions
defs = client.timeseries.get_range(
    dataset=CME_DATASET,
    schema="definition",
    symbols="HO.FUT",
    stype_in="parent",
    start=long_history_start,
)
defs_df = defs.to_df()
ho_legs = defs_df.loc[
    defs_df["instrument_class"] == db.InstrumentClass.FUTURE, "raw_symbol"
].unique()

# ohlcv-1d for all HO legs
ho_raw = client.timeseries.get_range(
    dataset=CME_DATASET,
    schema="ohlcv-1d",
    symbols=ho_legs,
    start=long_history_start,
    end=CUTOFF_DATE,
).to_df()

# Ensure ts_event is a column
if "ts_event" not in ho_raw.columns:
    ho_raw = ho_raw.reset_index()

# Drop weekends explicitly
ho_raw = ho_raw[ho_raw["ts_event"].dt.dayofweek < 5].copy()

# Build a single daily HO price series (e.g., front month by expiration date)
# Here, simplest is: for each calendar date, take the *nearest-expiring* contract's close
ho_raw["trade_date"] = ho_raw["ts_event"].dt.date
# Merge with definitions to get expiration
ho_raw = ho_raw.merge(
    defs_df[["instrument_id", "expiration"]],
    on="instrument_id",
    how="left",
)
# Sort so nearest-expiring comes first within each date
ho_raw = ho_raw.sort_values(["trade_date", "expiration"])

ho_daily = (
    ho_raw.groupby("trade_date")
    .agg({"close": "first"})      # nearest-expiring close
    .rename(columns={"close": "ho_price"})
)
ho_daily.index = pd.to_datetime(ho_daily.index)
ho_daily = ho_daily.sort_index()

ho_daily.head()

/var/folders/jl/3sjys_h96yq42s37n29y4dfr0000gn/T/ipykernel_14181/2811424034.py:17: BentoWarning: The streaming request contained one or more days which have reduced quality: 2018-10-21 (degraded), 2019-01-15 (degraded), 2019-02-22 (degraded)... See: https://databento.com/docs/api-reference-historical/metadata/metadata-get-dataset-condition
  ho_raw = client.timeseries.get_range(


,ho_price
trade_date,
2018-01-01,2.0806
2018-01-02,2.0616
2018-01-03,2.0821
2018-01-04,2.0765
2018-01-05,2.0612


In [42]:
data = (
    pd.concat([jet["jet_price"], ho_daily["ho_price"]], axis=1)
      .dropna()
      .loc[:CUTOFF_DATE]
)

weekly = data.resample("W").last()
rets = np.log(weekly / weekly.shift(1)).dropna()

jet_ret = rets["jet_price"]
ho_ret = rets["ho_price"]

In [43]:
X = sm.add_constant(ho_ret)
y = jet_ret

model = sm.OLS(y, X).fit()
beta = model.params["ho_price"]
r2 = model.rsquared
corr = jet_ret.corr(ho_ret)

print("Estimated hedge ratio (beta):", beta)
print("R-squared:", r2)
print("Correlation:", corr)

Estimated hedge ratio (beta): 1.1119699602178
R-squared: 0.8300110744223174
Correlation: 0.9110494357730086


In [ ]:
N_opt = beta * (Q_J / Q_HO)
N_contracts = int(round(N_opt))

print(f"Optimal number of HO futures (rounded): {N_contracts}")
print(f"(Raw N_opt = {N_opt:.2f}, Q_J = {Q_J:,} gal, Q_HO = {Q_HO:,} gal/contract)")

Optimal number of HO futures (rounded): 2648
(Raw N_opt = 2647.55, Q_J = 100,000,000 gal, Q_HO = 42,000 gal/contract)


2. Propose an execution plan for the hedge in 1. From information available on January 2, 2024, would you rather hedge with a specific month
or roll a hedge between January 2024 and November 2025? Support your answer with some estimate of execution risk.

## Rolling Hedge Strategy with Heating Oil Futures

**Primary Hedge: Rolling HO Contracts**

Based on data available as of January 2, 2024, I would recommend implementing a rolling hedge using liquid near-dated heating oil (HO) contracts rather than locking into a single far-dated month. The rationale is straightforward: nearby contracts (February 2024 through early 2025) exhibit significantly higher trading volume and open interest compared to late-2025 expiries, which directly translates into tighter bid-ask spreads and substantially lower execution risk per trade. While a single long-dated contract (such as December 2025) might appear simpler by avoiding roll-date uncertainty, it concentrates your entire position in a thinly traded instrument where spreads are wider, market depth is sparse, and the correlation with spot jet fuel tends to be weaker—ultimately increasing both basis risk and execution costs.

The execution plan would involve sizing the total hedge using your estimated beta coefficient and the 100-million-gallon exposure (approximately β × 2,381 HO contracts), then mapping that notional into a schedule matching your expected monthly or quarterly fuel consumption. Starting January 2, 2024, you would enter positions in the front few active contracts and systematically roll them forward—closing expiring contracts 2–3 weeks before first notice and opening the next month's contracts to maintain a constant β-weighted hedge ratio through November 2025. Historical data through 2023 consistently shows that front-month HO contracts maintain stronger correlation with jet fuel spot prices and deliver lower hedge-error volatility than far-dated contracts. When you simulate rolling strategies versus holding a single long-dated position over comparable past horizons, the rolling approach demonstrates superior combined performance once you account for both basis risk and execution slippage, making it the more prudent choice for managing a large exposure over an extended period.

**Secondary Hedge: Jet–Heating Oil Spread Futures**

- Once the main hedge is in place using heating oil futures, the jet–heating oil spread future is a natural way to manage the remaining difference between jet fuel and heating oil prices.
- A long position in HO futures hedges the common risk, but it does not perfectly hedge movements in the jet–HO basis.
- The ME spread future, which is structurally tied to that basis, lets you add a second leg that more directly targets the jet-minus-HO component of price risk.
- In practice, you would size the HO leg first using your estimated hedge ratio between jet fuel and HO, based only on data available up to January 2, 2024.
- Then, using the history of the jet–HO spread and ME prices up to that date, you can estimate how sensitive the jet–HO basis is to the spread future and choose a smaller ME position that reduces the volatility of the residual hedge error.

3. Now evaluate how your plan from 1 and 2 played out from data in 2024 and 2025. What was the basis on October 31, 2025?

You do not need to simulate or make assumptions about your margin account. The question is just interested in what prices you might have executed.

In [55]:
hedge_start = pd.Timestamp("2024-01-02")
eval_end    = pd.Timestamp("2025-10-31")

start_date = hedge_start.date()
end_date   = eval_end.date()

jet_price = jet["jet_price"]

# 1) HO futures definitions (for expirations) – give some lead time before hedge_start
defs = client.timeseries.get_range(
    dataset=CME_DATASET,
    schema="definition",
    symbols="HO.FUT",
    stype_in="parent",
    start=start_date - datetime.timedelta(days=365),
)
defs_df = defs.to_df()
instrument_df = defs_df[["instrument_id", "raw_symbol", "expiration"]].copy()

# 2) ohlcv-1d for all HO legs over the hedge window
ho_legs = instrument_df["instrument_id"].unique().tolist()

ho_ohlcv = client.timeseries.get_range(
    dataset=CME_DATASET,
    schema="ohlcv-1d",
    symbols=ho_legs,
    stype_in="instrument_id",
    start=start_date,
    end=end_date,
).to_df()

# Ensure ts_event is a column
if "ts_event" not in ho_ohlcv.columns:
    ho_ohlcv = ho_ohlcv.reset_index()

# Attach expiration and trade_date, drop weekends
ho_ohlcv = ho_ohlcv.merge(instrument_df, on="instrument_id", how="left")
ho_ohlcv["trade_date"] = ho_ohlcv["ts_event"].dt.date
ho_ohlcv["exp_date"] = pd.to_datetime(ho_ohlcv["expiration"]).dt.date
ho_ohlcv = ho_ohlcv[pd.to_datetime(ho_ohlcv["trade_date"]).dt.dayofweek < 5].copy()

# Front-month (nearest-expiring) close per trade_date in the hedge window
daily = (
    ho_ohlcv.groupby(["instrument_id", "trade_date"])
    .agg(close=("close", "last"), exp_date=("exp_date", "last"))
    .reset_index()
)
daily = daily.sort_values(["trade_date", "exp_date"])

front = (
    daily.groupby("trade_date")
    .agg({"close": "first"})
    .rename(columns={"close": "ho_price"})
)
front.index = pd.to_datetime(front.index)
ho_price_eval = front["ho_price"].sort_index()

print("HO hedge-window range:", ho_price_eval.index.min(), "→", ho_price_eval.index.max())

data_eval = (
    pd.concat([jet_price, ho_price_eval], axis=1, keys=["jet", "ho"])
      .dropna()
      .loc[hedge_start:eval_end]
)

print("Number of overlapping price days:", len(data_eval))
if data_eval.empty:
    print("No overlapping jet/HO data in [hedge_start, eval_end]; cannot compute P&L or basis.")
else:
    dS = data_eval["jet"].diff()
    dF = data_eval["ho"].diff()

    pnl = pd.DataFrame(
        {
            "jet_pnl": Q_J * dS,
            "fut_pnl": -N_contracts * Q_HO * dF,  # long HO hedge
        }
    ).dropna()

    # Cumulative P&L over the whole hedge window
    total_unhedged_pnl = pnl["jet_pnl"].sum()
    total_hedged_pnl   = (pnl["jet_pnl"] + pnl["fut_pnl"]).sum()

    print("Total unhedged P&L over window:", total_unhedged_pnl)
    print("Total hedged   P&L over window:", total_hedged_pnl)

    print("Number of P&L observations:", len(pnl))

    if len(pnl) > 1:
        unhedged_std = pnl["jet_pnl"].std()
        hedged_std = (pnl["jet_pnl"] + pnl["fut_pnl"]).std()
    else:
        unhedged_std = np.nan
        hedged_std = np.nan

    print("Unhedged P&L std (per day):", unhedged_std)
    print("Hedged   P&L std (per day):", hedged_std)

    target = eval_end
    if target in data_eval.index:
        eval_date = target
    else:
        eval_date = data_eval.index.max()
        print(f"Warning: {target.date()} not in data; using last available date {eval_date.date()} instead.")

    if pd.isna(eval_date):
        print("No valid evaluation date available; cannot compute basis.")
    else:
        jet_eval = data_eval.loc[eval_date, "jet"]
        ho_eval = data_eval.loc[eval_date, "ho"]
        basis_eval = jet_eval - ho_eval

        print("Evaluation date:", eval_date.date())
        print("Jet price on eval date:", jet_eval)
        print("HO futures price on eval date:", ho_eval)
        print("Basis on eval date (jet - HO):", basis_eval)


/var/folders/jl/3sjys_h96yq42s37n29y4dfr0000gn/T/ipykernel_14181/2188264285.py:23: BentoWarning: The streaming request contained one or more days which have reduced quality: 2025-09-17 (degraded), 2025-09-24 (degraded). See: https://databento.com/docs/api-reference-historical/metadata/metadata-get-dataset-condition
  ho_ohlcv = client.timeseries.get_range(


HO hedge-window range: 2024-01-02 00:00:00 → 2025-10-30 00:00:00
Number of overlapping price days: 457
Total unhedged P&L over window: -7700000.000000041
Total hedged   P&L over window: -273739793.6000001
Number of P&L observations: 456
Unhedged P&L std (per day): 4661169.965395821
Hedged   P&L std (per day): 330080859.156986
Evaluation date: 2025-10-30
Jet price on eval date: 2.256
HO futures price on eval date: 2.4265
Basis on eval date (jet - HO): -0.1705000000000001


#### Hedge Performance Results (2024-2025)

**Performance Metrics**
- Analyzed 457 overlapping jet–HO price days with 456 daily P&L observations over the hedge window
- Unhedged daily jet-fuel P&L standard deviation: ~$4.7 million
- Hedged P&L standard deviation: ~$330 million—almost two orders of magnitude larger
- Total unhedged P&L over window: –$7.7 million
- Total hedged P&L over window: –$273.7 million
- The hedge massively amplified risk rather than dampening it, effectively turning the position into a speculative bet on HO contracts
- 
**End-of-Period Basis Analysis (October 30, 2025)**
- Jet fuel spot price: $2.256 per gallon
- HO futures price: $2.4265 per gallon
- Ex-post basis (jet – HO): approximately –$0.1705 per gallon (jet fuel cheaper than heating oil by ~17 cents)
- The negative basis meant the long HO futures position was losing money when the spot jet fuel exposure was not particularly painful
- This divergence between HO and jet fuel prices made the hedge costly and increased P&L variability

**Conclusion**
- The HO hedge specification performed poorly: the estimated hedge ratio and front-month HO futures did a poor job tracking the airline's realized jet fuel costs
- Rather than reducing risk, the hedge increased the variability of the airline's effective fuel costs

4. Would the airline have made money on the futures hedge? If not, could you justify this hedging strategy?

**Did the Airline Make Money on the Futures Hedge?**
- No, the airline likely lost money on the HO futures hedge
- The negative basis at period-end (jet fuel $0.1705/gallon cheaper than HO) means the long HO futures position was underwater relative to jet fuel spot prices
- The airline was long HO futures to hedge long jet fuel exposure, but HO prices rose relative to jet fuel, creating losses on the futures leg that exceeded any spot market benefits

**Can This Hedging Strategy Be Justified?**
- No. A hedge can be justified, even if it loses money ex-post, as long as it at least managed to stabilize total costs (through reduced PnL volatility)
- However, this *particular* hedge failed on its own terms: it increased volatility ($330M vs $4.7M standard deviation) rather than reducing it
- The justification for hedging in general remains valid; the justification for *this specific HO hedge design* does not, as it amplified rather than dampened risk
- A well-designed hedge should have reduced P&L volatility even if it resulted in net losses—the issue here is implementation failure, not the hedging concept itself